In [1]:
### Import libraries
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn import cross_validation
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
%pylab
from sklearn.pipeline import make_pipeline
from scipy.signal import argrelextrema
import statsmodels.discrete.discrete_model as sm

/home/deva/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [2]:
df_outliers = pd.read_excel('DistrictTypes.xlsx')

In [3]:
df_outliers.head()

,District,State,Type,DistCode
0,Agra,Uttar Pradesh,4. Large City,146
1,Ahmadabad,Gujarat,1. Metro,474
2,Aizawl,Mizoram,3. Capital,283
3,Ajmer,Rajasthan,4. Large City,119
4,Allahabad,Uttar Pradesh,4. Large City,175


In [4]:
snow_id = df_outliers[df_outliers['Type'] == '5. Snow Clad']['DistCode'].values.tolist()

In [5]:
metro = df_outliers[df_outliers['Type'] == '1. Metro']['DistCode'].values.tolist()

In [6]:
### Import Data Files
df_satData = pd.read_csv('distSatellite_withLabels.csv')

In [16]:
df_asset = pd.get_dummies(pd.read_csv('District_HHD_Cluster_Asset_Labels.csv')[['District','District_HHD_Cluster_Asset']])

In [18]:
df_asset.columns = ['dist_code','asset_1','asset_2','asset_3']

In [23]:
df_satData = (df_satData.merge(df_asset,left_on='101',right_on='dist_code',how='outer'))

In [24]:
del df_satData['dist_code']

In [ ]:
df_satData = df_satData[~(df_satData['101'].isin(snow_id+metro))]

In [26]:
### List of labels
lis_labelsAll = ['MSL_1','MSL_2','MSL_3','MSW_1','MSW_2','MSW_3','CHH_1','CHH_2','CHH_3','FC_1','FC_2','FC_3','BF_1','BF_2','BF_3','EMP_1','EMP_2','EMP_3']

### List of useful Labels
lis_labels= ['asset_1','asset_2','asset_3']

In [27]:
df_satData.head()

,DISTRICT,101,DN_00,DN_01,DN_02,DN_03,DN_04,DN_05,DN_06,DN_07,...,FC_3,BF_1,BF_2,BF_3,EMP_1,EMP_2,EMP_3,asset_1,asset_2,asset_3
0,Adilabad,532,5237.12,0,0,0.76,1815.67,3332.38,2043.90,1049.05,...,0,1,0,0,0,1,0,0,1,0
1,Agra,146,401.66,0,0,0.00,48.54,451.36,620.77,441.57,...,1,0,0,1,1,0,0,0,1,0
2,Ahmadabad,474,1517.13,0,0,14.19,1274.94,1784.85,816.68,461.02,...,1,0,0,1,0,0,1,0,0,1
3,Ahmadnagar,522,984.52,0,0,3.12,1756.19,3855.19,2683.53,1989.69,...,0,1,0,0,0,1,0,0,1,0
4,Aizawl,283,3412.59,0,0,0.00,49.31,102.02,42.99,27.32,...,1,0,0,1,0,0,1,0,0,1


In [28]:
df_satData.shape

(640, 106)

In [29]:
cols = df_satData.columns.tolist()

In [30]:
cols_new = cols[2:cols.index('MSL_1')]

In [32]:
for i,quantile_val in enumerate(quantile_list):
    df_satData.loc[df_satData[cols_new[i]] > quantile_val,cols_new[i]] = quantile_val

NameError: name 'quantile_list' is not defined

In [33]:
pure_modis_features = cols[cols.index('Water'):cols.index('MSL_1')]

## Building features from night light data

In [35]:
var = []
for i in range(64):
    var.append('DN_' + str(i).zfill(2))

df_satData['Area'] = 0
for i in var:
    df_satData['Area']=df_satData['Area'] + df_satData[i]
df_satData['Area'] = df_satData['Area'].replace(0,1)

for i in var:
    df_satData['p'+i] = (df_satData[i])/(df_satData['Area']*1.0)

for i,j in enumerate(var):
    df_satData['q'+j] = df_satData[j]*(i+1)

var3 = []
for i in var:
    var3.append('p'+i)
for i in var:
    var3.append('q'+i)

## Appling Pca to matrix formed from features of nightlight data

In [36]:
X = df_satData.loc[:,var3].values

In [37]:
X = preprocessing.scale(X)
pca = PCA()
pca.fit(X)

#pca= PCA(svd_solver='full')
#pca.fit(X, y=None )

#The amount of variance that each PC explains
var= pca.explained_variance_ratio_

#Cumulative Variance explains
var1=np.cumsum(np.round(pca.explained_variance_ratio_, decimals=4)*100)
print(var1)

#Number of PC comprising 95%Variance
n = len(var1)-len([num for num in var1 if num != 0 and num > 85.0])+1
print( "n: 95% Variance: ",n)

[ 43.38  60.43  68.59  72.97  76.31  78.6   80.8   82.77  84.32  85.64
  86.87  87.94  88.82  89.59  90.26  90.89  91.43  91.89  92.33  92.74
  93.12  93.45  93.76  94.05  94.32  94.58  94.81  95.03  95.24  95.44
  95.64  95.83  96.01  96.18  96.35  96.51  96.67  96.82  96.96  97.1
  97.23  97.36  97.48  97.59  97.7   97.81  97.91  98.01  98.11  98.2
  98.29  98.38  98.47  98.55  98.63  98.7   98.77  98.84  98.91  98.98
  99.04  99.1   99.16  99.21  99.26  99.31  99.36  99.4   99.44  99.48
  99.51  99.54  99.57  99.6   99.63  99.66  99.68  99.7   99.72  99.74
  99.76  99.78  99.79  99.8   99.81  99.82  99.83  99.84  99.85  99.86
  99.87  99.88  99.89  99.9   99.91  99.92  99.93  99.94  99.95  99.96
  99.96  99.96  99.96  99.96  99.96  99.96  99.96  99.96  99.96  99.96
  99.96  99.96  99.96  99.96  99.96  99.96  99.96  99.96  99.96  99.96
  99.96  99.96  99.96  99.96  99.96  99.96  99.96  99.96]
('n: 95% Variance: ', 10)


In [38]:
### PCA fit
X=pca.fit_transform(X)
X=X[:,0:n]

In [39]:
n

10

In [ ]:
## creating variance table 

In [ ]:
df_pcaComp = pd.DataFrame(pca.components_)

In [ ]:
pca_idx = ['PCA_'+str(i) for i in range(len(df_pcaComp))]

In [ ]:
df_pcaComp.index = pca_idx

In [ ]:
df_pcaComp.columns = var3

In [ ]:
abs(df_pcaComp[column][:10])/abs(df_pcaComp[:10]).sum(axis=1)

In [ ]:
bins_pcaVar = [[0]*10]*7

for column in df_pcaComp:
    bins_pcaVar[(int(column[4:])/10)] += (abs(df_pcaComp[column][:10])/abs(df_pcaComp[:10]).sum(axis=1))

bins_pcaVar[5] += bins_pcaVar[6]

In [ ]:
df_bins_pcaVar = pd.DataFrame(bins_pcaVar[:6]).T

In [ ]:
df_bins_pcaVar.columns = ['L1','L2','M1','M2','H1','H2']

In [ ]:
df_bins_pcaVar

In [ ]:
df_newVarTable = pd.DataFrame(columns=['0','1','2','3','4','5'])

for i in range(10):
    temp = df_bins_pcaVar[i:i+1].sort_values(by=pca_idx[i],axis=1,ascending=False)
    temp_cols = temp.columns.tolist()
    temp_vals = np.round(temp.values[0],2).tolist()
    new_row = [str(temp_vals[j])+','+temp_cols[j] for j in range(len(temp_cols))]
    df_newVarTable.loc[pca_idx[i],:] = new_row
    

In [ ]:
#df_newVarTable.to_csv('newVarTable.csv')

In [40]:
### Creating Dataframe of PCA
col_X = []
for j in range(n):
    col_X.append('PCA_'+str(j))
df_pcaNTL = pd.DataFrame(X,columns=col_X)

In [41]:
df_pcaNTL['101'] = df_satData['101'].values.tolist()

In [42]:
df_pcaNTL.head()

,PCA_0,PCA_1,PCA_2,PCA_3,PCA_4,PCA_5,PCA_6,PCA_7,PCA_8,PCA_9,101
0,2.730040,-5.321751,-0.860339,3.246664,2.694576,0.809633,1.399437,-1.985949,0.947422,0.560823,532
1,8.599441,-1.433546,-0.258951,-0.041424,-0.715240,0.191611,-1.324506,1.446709,-0.833139,0.085463,146
2,12.113937,-4.310023,-7.136619,-2.049438,1.315599,1.148854,-0.142953,-1.886010,-3.099527,-1.958706,474
3,9.380082,-9.838996,1.143180,7.110819,1.978624,0.743656,3.220930,-1.526049,1.006662,0.924702,522
4,-4.989618,0.675239,-0.595747,-1.363547,-0.947617,-0.620833,0.649545,0.154708,0.866108,-0.017051,283


## Building features out of Modis Data

In [43]:
df_satData['Urban and built-up']=df_satData['Urban and built-up']+1
df_satData['CropRatio']=(df_satData['Croplands']+df_satData['Cropland/Natural vegetation mosaic'])/df_satData['Area']
df_satData['AvgUrbanNTL']=df_satData['sum']/df_satData['Urban and built-up']
df_satData['UrbanRatio']= df_satData['Urban and built-up']/df_satData['Area']

df_satData['Natural']= 0
for i in ['Water','Evergreen Needleleaf forest','Evergreen Broadleaf forest','Deciduous Needleleaf forest','Deciduous Broadleaf forest','Mixed forest','Closed shrublands','Open shrublands','Woody savannas','Savannas','Grasslands','Permanent wetlands','Snow and ice']:
    df_satData['Natural']=df_satData['Natural']+df_satData[i]
    
df_satData['CropRemainRatio']=(df_satData['Croplands']+df_satData['Cropland/Natural vegetation mosaic'])/(df_satData['Area']-df_satData['Natural'])
df_satData['UrbanRemainRatio']=df_satData['Urban and built-up']/(df_satData['Area']-df_satData['Natural'])
#df_satData['UrbanRatio']=df_satData['UrbanRatio'].fillna(0)


forest =['Evergreen Broadleaf forest','Deciduous Broadleaf forest','Mixed forest']
df_satData['forest'] = df_satData[forest].sum(axis=1)

grass_shrubs =['Closed shrublands','Open shrublands','Woody savannas','Savannas','Grasslands']
df_satData['grass_shrubs'] = df_satData[grass_shrubs].sum(axis=1)


In [44]:
modi_area = df_satData[pure_modis_features].sum(axis=1).values.tolist()

In [ ]:
left_area = [df_satData['Area'].values.tolist()[i]-modi_area[i] for i in range(len(df_satData))]

In [ ]:
df_satData['left_area'] = left_area

In [45]:
modi_var = []
modi_var=['mean','sum','CropRatio', 'AvgUrbanNTL', 'UrbanRatio', 'CropRemainRatio', 'UrbanRemainRatio','Natural']
for i in ['Croplands',
 'Urban and built-up',
 'Cropland/Natural vegetation mosaic',
 'forest',
 'grass_shrubs']:
    df_satData['mod_'+i]=df_satData[i]/df_satData['Area']
    modi_var.append('mod_'+i)
    modi_var.append(i)
#modi_var.append('left_area')

## final dataframe from pca version of nl and modi 

In [46]:
df_modi_nlPca =  pd.merge(df_satData[['101']+modi_var],df_pcaNTL,left_on='101',right_on='101',how='inner')

In [47]:
df_modi_nlPca.shape

(640, 29)

In [48]:
var = df_modi_nlPca.columns.tolist()[1:]

## finding feature ranking for a particular label

In [50]:
def featureRank(label):
    model = LogisticRegression()
    selector = RFE(model,1, step=1)

    X = df_modi_nlPca[var].values
    X = preprocessing.scale(X)

    y = df_satData.loc[:,label].values

    selector.fit(X,y)

    selectedIdx = selector.get_support(indices=True)

    featureRanking = selector.ranking_

    ranked_var = pd.DataFrame(featureRanking,var)
    ranked_var = ranked_var.reset_index()
    ranked_var.columns = (['var','rank'])
    ranked_var = ranked_var.sort_values('rank')
    ranked_var.reset_index(inplace=True)
    del ranked_var['index']
    
    return ranked_var

## selecting features to maximize f1_score
#### Assumption: Set of first n features in feature ranking will produce maximum f1_score compared to any other set of n features.

In [51]:
def selectFeatures(ranked_var,label):
    metric_list = list()
    f1_list = list()
    for i in range(len(ranked_var)):
        var2 = ranked_var.loc[:i,'var'].values.tolist()

        X = df_modi_nlPca.loc[:,var2].values
        y = df_satData.loc[:,label].values

        model = LogisticRegression(class_weight='balanced')
        clf = make_pipeline(preprocessing.StandardScaler(),model)
        predicted = cross_validation.cross_val_predict(clf,X,y, cv=5)

        acc = metrics.accuracy_score(y,predicted)
        recall = metrics.recall_score(y,predicted)
        prec =  metrics.precision_score(y,predicted)

        cm = metrics.confusion_matrix(y, predicted)
        cm_score = float(cm[1][1])/(cm[1][1]+cm[1][0]+cm[0][1])

        f1 = metrics.f1_score(y,predicted)

        metric_list.append([f1,cm_score,acc,recall,prec])
        f1_list.append(f1)

    df_metric = pd.DataFrame(metric_list)
    df_metric.columns =  (['f1','cm','acc','recall','prec'])

    ## finding the global maximum index in f1_list
    global_maxima = f1_list.index(max(f1_list))

    ## plotting f1_score with features selected
    #figure()
    #plt.grid()
    #plt.plot(f1_list)
    #plt.title(label)
    #plt.xlabel('number of variables')
    #plt.ylabel('f1_score')
    
    ## findi  ng indexes of all local maximums

    x = array(f1_list)
    # for local maxima
    local_maxima = argrelextrema(x, np.greater)[0].tolist()

    f1_acceptable = f1_list[global_maxima]-0.01

    i = 0
    while f1_list[i] < f1_acceptable:
        i += 1

    var_final = ranked_var.loc[:i,'var']

    X = df_modi_nlPca.loc[:,var_final].values
    y = df_satData.loc[:,label].values

    model = LogisticRegression()

    fit = model.fit(X,y)

    df_selecedVar = ranked_var.loc[:i].copy(deep=True)
    df_selecedVar.insert(2,'coeff',fit.coef_[0])

    return [df_selecedVar,f1_list[i],df_metric[i:i+1]]

In [52]:
select_feature = []
score_metric = []
for label in lis_labels:
    ranked_var = featureRank(label)
    feature_and_score = selectFeatures(ranked_var,label)
    select_feature.append(feature_and_score)
    score_metric.append(feature_and_score[2].values[0])

/home/deva/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/data.py:164: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "


In [54]:
df_scoreMetric = pd.DataFrame(score_metric,columns = ['f1','cm','acc','recall','prec'], index=lis_labels)

In [57]:
df_scoreMetric.to_csv('scoreMetric_asset.csv')

In [56]:
df_scoreMetric.sort_values(by='f1',ascending=False)

,f1,cm,acc,recall,prec
asset_1,0.791531,0.654987,0.800000,0.883636,0.716814
asset_2,0.679054,0.514066,0.703125,0.681356,0.676768
asset_3,0.655738,0.487805,0.901563,0.857143,0.530973


In [ ]:
select_feature[0][0]

In [ ]:
## adding pvalues.... to the list of featues.

In [ ]:
X = df_modi_nlPca.copy(deep=True)
X.insert(1,'intercept',1.0)
cols = X.columns.tolist()

In [ ]:
modi_cols = cols[1:cols.index('PCA_0')]
pca_cols = ['intercept'] + cols[cols.index('PCA_0'):]

In [ ]:
def find_summary(cols,label):
    X1 = X[cols].values
    y = df_satData.loc[:,label].values

    model = sm.Logit(y,X1)
    result = model.fit()
    summary = result.summary()

    df_summary = pd.DataFrame(result.params,result.pvalues)
    df_summary.reset_index(inplace=True)
    df_summary.index = cols
    df_summary.columns = ['pvals','param']
    
    return df_summary

In [ ]:
selectedFeatures_pval = []
for i,label in enumerate(lis_labelsAll):
    df_temp = select_feature[i][0].copy(deep=True)
    df_temp.index = df_temp['var']
    del df_temp['var']
    df_summary = pd.concat([find_summary(modi_cols,label),find_summary(pca_cols,label)])
    df_temp2 = pd.concat([df_temp,df_summary],axis=1,join_axes=[df_temp.index])
    selectedFeatures_pval.append(df_temp2)

In [ ]:
label = 'BF_1'
selectedFeatures_pval[lis_labelsAll.index(label)]